In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import nltk
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score
import datasets
import joblib
import os
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Grammar_autocorrection_df1_new.csv')

In [4]:
df.head()

,Error Type,Ungrammatical Statement,Standard English,Ungrammatical Length,Standard Length,Tokenized Ungrammatical,Tokenized Standard,POS_Ungrammatical,POS_Standard,POS_W2V_Ungram,...,W2V_Ungram,W2V_Standard,Bigrams_Ungrammatical,Bigrams_Standard,Trigrams_Ungrammatical,Trigrams_Standard,W2V_Bigrams_Ungrammatical,W2V_Bigrams_Standard,W2V_Trigrams_Ungrammatical,W2V_Trigrams_Standard
0,verb tense errors,i goes to the store everyday,i go to the store everyday,29,27,"['i', 'goes', 'to', 'the', 'store', 'everyday']","['i', 'go', 'to', 'the', 'store', 'everyday']","[('i', 'NN'), ('goes', 'VBZ'), ('to', 'TO'), (...","[('i', 'NN'), ('go', 'VBP'), ('to', 'TO'), ('t...",[-0.02586105 -0.0514316 0.06859668 0.061394...,...,[-0.32014304 0.31378677 0.17460077 0.115764...,[-0.3323602 0.32704362 0.17878683 0.124437...,"['i goes', 'goes to', 'to the', 'the store', '...","['i go', 'go to', 'to the', 'the store', 'stor...","['i goes to', 'goes to the', 'to the store', '...","['i go to', 'go to the', 'to the store', 'the ...",[-0.00710673 0.01571953 0.02464187 0.003107...,[-0.00055286 0.01658004 0.03385994 0.003596...,[ 0.01779238 -0.00231509 -0.00103101 -0.004174...,[ 0.01239582 -0.00447729 -0.00167276 0.005272...
1,verb tense errors,they was playing soccer last night,they were playing soccer last night,35,36,"['they', 'was', 'playing', 'soccer', 'last', '...","['they', 'were', 'playing', 'soccer', 'last', ...","[('they', 'PRP'), ('was', 'VBD'), ('playing', ...","[('they', 'PRP'), ('were', 'VBD'), ('playing',...",[ 0.06836 -0.11103898 -0.09156247 0.026392...,...,[-3.74800593e-01 3.41475725e-01 2.17105433e-...,[-3.64397138e-01 3.43511343e-01 2.18918696e-...,"['they was', 'was playing', 'playing soccer', ...","['they were', 'were playing', 'playing soccer'...","['they was playing', 'was playing soccer', 'pl...","['they were playing', 'were playing soccer', '...",[-2.09984952e-03 5.61572425e-03 1.10909315e-...,[ 4.7441646e-03 8.7659908e-03 3.8178433e-03 ...,[-2.4519933e-03 -8.5647805e-03 7.7129027e-04 ...,[ 2.1319385e-03 -5.1139006e-03 -3.4539052e-03 ...
2,verb tense errors,she have completed her homework,she has completed her homework,32,31,"['she', 'have', 'completed', 'her', 'homework']","['she', 'has', 'completed', 'her', 'homework']","[('she', 'PRP'), ('have', 'VBP'), ('completed'...","[('she', 'PRP'), ('has', 'VBZ'), ('completed',...",[-0.07061146 -0.13248616 -0.13986623 0.036880...,...,[-0.3181368 0.3165172 0.16193333 0.103851...,[-3.3535850e-01 3.1291074e-01 1.7064826e-01 ...,"['she have', 'have completed', 'completed her'...","['she has', 'has completed', 'completed her', ...","['she have completed', 'have completed her', '...","['she has completed', 'has completed her', 'co...",[ 0.00564092 0.00151694 0.00889045 0.003766...,[ 0.0003912 -0.00584303 0.01212654 0.003562...,[ 0.00491791 -0.00564554 -0.00283222 0.004411...,[-0.01037725 -0.00692834 -0.00052899 0.011143...
3,verb tense errors,he do not know the answer,he does not know the answer,25,27,"['he', 'do', 'not', 'know', 'the', 'answer']","['he', 'does', 'not', 'know', 'the', 'answer']","[('he', 'PRP'), ('do', 'VBZ'), ('not', 'RB'), ...","[('he', 'PRP'), ('does', 'VBZ'), ('not', 'RB')...",[-0.13194092 -0.07158052 -0.15803912 -0.055573...,...,[-3.76218051e-01 3.76779526e-01 1.89181253e-...,[-0.35672054 0.35583436 0.17960103 0.117476...,"['he do', 'do not', 'not know', 'know the', 't...","['he does', 'does not', 'not know', 'know the'...","['he do not', 'do not know', 'not know the', '...","['he does not', 'does not know', 'not know the...",[ 1.7322022e-04 2.3502617e-03 1.3704985e-02 ...,[-3.8101412e-03 -4.7947731e-04 9.8749576e-03 ...,[ 1.1982820e-03 -2.2560721e-03 -1.9308180e-04 ...,[ 4.0928507e-03 -9.6415197e-03 5.6834938e-03 ...
4,verb tense errors,the sun rise in the east,the sun rises in the east,25,26,"['the', 'sun', 'rise', 'in', 'the', 'east']","['the', 'sun', 'rises', 'in', 'the', 'east']","[('the', 'DT'), ('sun', 'NN'), ('rise', 'NN'),...","[('the', 'DT'), ('sun', 'NN'), ('rises',

In [5]:
# printing dataset columns
print(df.columns)

Index(['Error Type', 'Ungrammatical Statement', 'Standard English',
       'Ungrammatical Length', 'Standard Length', 'Tokenized Ungrammatical',
       'Tokenized Standard', 'POS_Ungrammatical', 'POS_Standard',
       'POS_W2V_Ungram', 'POS_W2V_Standard', 'POS_seq_U', 'POS_seq_S',
       'W2V_Ungram', 'W2V_Standard', 'Bigrams_Ungrammatical',
       'Bigrams_Standard', 'Trigrams_Ungrammatical', 'Trigrams_Standard',
       'W2V_Bigrams_Ungrammatical', 'W2V_Bigrams_Standard',
       'W2V_Trigrams_Ungrammatical', 'W2V_Trigrams_Standard'],
      dtype='object')


# Naive Bayes

In [6]:
embedding_experiments = {
    'POS':     ('POS_W2V_Ungram',             'POS_W2V_Standard'),
    'Unigram': ('W2V_Ungram',                 'W2V_Standard'),
    'Bigram':  ('W2V_Bigrams_Ungrammatical',  'W2V_Bigrams_Standard'),
    'Trigram': ('W2V_Trigrams_Ungrammatical', 'W2V_Trigrams_Standard')
}

In [7]:
# ── 0) Define & apply your parser once ──────────────────────────────────────
def parse_embedding(s):
    if isinstance(s, str):
        return np.array([float(x) for x in s.strip('[]').split()], dtype=float)
    return np.zeros(0, dtype=float)

for ug_col, st_col in embedding_experiments.values():
    df[ug_col] = df[ug_col].apply(parse_embedding)
    df[st_col] = df[st_col].apply(parse_embedding)

# ── 1) Pipeline builder ─────────────────────────────────────────────────────
def make_embedding_pipeline(ug_col, st_col):
    n = len(df)
    # Stack the two columns into one long list of arrays
    feats = list(df[ug_col]) + list(df[st_col])
    y     = np.array([0]*n + [1]*n)               # 0=ungrammatical, 1=correct
    X_df  = pd.DataFrame({'feat': feats})

    # transformer: np.vstack on that 'feat' column → (2n × dim) matrix
    stacker = FunctionTransformer(lambda X: np.vstack(X['feat']), validate=False)

    pipe = Pipeline([
        ('stack', stacker),
        ('clf',   GaussianNB())
    ])
    return pipe, X_df, y

# ── 2) Train/Test split evaluation (90/10) ─────────────────────────────────────────
print("=== Train/Test split 90/10 ===")
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline(ug, st)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X_df, y,
        test_size=0.1,
        stratify=y,
        random_state=42
    )
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)

    print(f"\n{name} accuracy: {accuracy_score(y_te, preds):.4f}")
    print(classification_report(y_te, preds, digits=4))

# ── 3) 10-fold CV ────────────────────────────────────────────────────────────
print("\n=== 10-Fold CV ===")
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline(ug, st)

    scores = cross_val_score(
        pipe, X_df, y,
        cv=cv,
        scoring='accuracy'
    )
    print(f"{name} CV accuracy: {scores.mean():.4f} ± {scores.std():.4f}")


=== Train/Test split 90/10 ===

POS accuracy: 0.5223
              precision    recall  f1-score   support

           0     0.5290    0.4059    0.4594       202
           1     0.5181    0.6386    0.5721       202

    accuracy                         0.5223       404
   macro avg     0.5236    0.5223    0.5157       404
weighted avg     0.5236    0.5223    0.5157       404


Unigram accuracy: 0.5198
              precision    recall  f1-score   support

           0     0.5160    0.6386    0.5708       202
           1     0.5260    0.4010    0.4551       202

    accuracy                         0.5198       404
   macro avg     0.5210    0.5198    0.5129       404
weighted avg     0.5210    0.5198    0.5129       404


Bigram accuracy: 0.4901
              precision    recall  f1-score   support

           0     0.4931    0.7030    0.5796       202
           1     0.4828    0.2772    0.3522       202

    accuracy                         0.4901       404
   macro avg     0.4879 

#SVM

In [8]:
from sklearn.svm import SVC

# Make directory to save models
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)

def stack_feats(X):
    return np.vstack(X['feat'])

def make_embedding_pipeline_svm(ug_col, st_col):
    n = len(df)
    feats = list(df[ug_col]) + list(df[st_col])
    y     = np.array([0]*n + [1]*n)  # 0=ungrammatical, 1=correct
    X_df  = pd.DataFrame({'feat': feats})
    stacker = FunctionTransformer(stack_feats, validate=False)

    pipe = Pipeline([
        ('stack', stacker),
        ('scaler', StandardScaler()),
        ('clf',   SVC(kernel='linear',class_weight='balanced',probability=True))
    ])
    return pipe, X_df, y

trained_pipelines = {}
print("=== Train/Test split 90/10 with SVM ===")
for name, (ug, st) in embedding_experiments.items():
    pipe, X_df, y = make_embedding_pipeline_svm(ug, st)

    X_tr, X_te, y_tr, y_te = train_test_split(
        X_df, y,
        test_size=0.1,
        stratify=y,
        random_state=42
    )
    pipe.fit(X_tr, y_tr)
    preds = pipe.predict(X_te)
    trained_pipelines[name] = pipe

    print(f"\n{name} accuracy: {accuracy_score(y_te, preds):.4f}")
    print(classification_report(y_te, preds, digits=4))

    # Save model
    model_path = os.path.join(save_dir, f"{name}_svm_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved model to {model_path}")

=== Train/Test split 90/10 with SVM ===

POS accuracy: 0.5446
              precision    recall  f1-score   support

           0     0.5563    0.4406    0.4917       202
           1     0.5369    0.6485    0.5874       202

    accuracy                         0.5446       404
   macro avg     0.5466    0.5446    0.5396       404
weighted avg     0.5466    0.5446    0.5396       404

Saved model to saved_models/POS_svm_pipeline.pkl

Unigram accuracy: 0.5025
              precision    recall  f1-score   support

           0     0.5026    0.4851    0.4937       202
           1     0.5024    0.5198    0.5109       202

    accuracy                         0.5025       404
   macro avg     0.5025    0.5025    0.5023       404
weighted avg     0.5025    0.5025    0.5023       404

Saved model to saved_models/Unigram_svm_pipeline.pkl

Bigram accuracy: 0.5322
              precision    recall  f1-score   support

           0     0.5337    0.5099    0.5215       202
           1     0.530

In [9]:
print("\n=== 10-Fold Cross Validation with SVM  ===")
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

cv_results = {}

for name, (ug, st) in embedding_experiments.items():
    # Prepare input features and labels
    pipe, X_df, y = make_embedding_pipeline_svm(ug, st)

    # Store fold-wise accuracy
    fold_accuracies = []

    for fold, (train_idx, val_idx) in enumerate(cv.split(X_df, y)):
        X_train, X_val = X_df.iloc[train_idx], X_df.iloc[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        fold_model = Pipeline([
            ('stack', FunctionTransformer(stack_feats, validate=False)),
            ('clf', SVC(kernel='linear', class_weight='balanced', probability=True))
        ])
        fold_model.fit(X_train, y_train)
        preds = fold_model.predict(X_val)
        acc = accuracy_score(y_val, preds)
        fold_accuracies.append(acc)
        print(f"{name} - Fold {fold+1} Accuracy: {acc:.4f}")

    # Summary statistics
    mean_acc = np.mean(fold_accuracies)
    std_acc = np.std(fold_accuracies)
    print(f"{name} - CV Accuracy: {mean_acc:.4f} ± {std_acc:.4f}")

    # Save the final model trained on the entire dataset
    pipe.fit(X_df, y)
    model_path = os.path.join(save_dir, f"{name}_CV_svm_pipeline.pkl")
    joblib.dump(pipe, model_path)
    print(f"Saved final CV model to {model_path}")


=== 10-Fold Cross Validation with SVM  ===
POS - Fold 1 Accuracy: 0.5495
POS - Fold 2 Accuracy: 0.5545
POS - Fold 3 Accuracy: 0.5619
POS - Fold 4 Accuracy: 0.4901
POS - Fold 5 Accuracy: 0.5272
POS - Fold 6 Accuracy: 0.5272
POS - Fold 7 Accuracy: 0.5186
POS - Fold 8 Accuracy: 0.4963
POS - Fold 9 Accuracy: 0.5062
POS - Fold 10 Accuracy: 0.4764
POS - CV Accuracy: 0.5208 ± 0.0273
Saved final CV model to saved_models/POS_CV_svm_pipeline.pkl
Unigram - Fold 1 Accuracy: 0.5198
Unigram - Fold 2 Accuracy: 0.4851
Unigram - Fold 3 Accuracy: 0.5668
Unigram - Fold 4 Accuracy: 0.5470
Unigram - Fold 5 Accuracy: 0.5322
Unigram - Fold 6 Accuracy: 0.4703
Unigram - Fold 7 Accuracy: 0.5459
Unigram - Fold 8 Accuracy: 0.5161
Unigram - Fold 9 Accuracy: 0.4988
Unigram - Fold 10 Accuracy: 0.5236
Unigram - CV Accuracy: 0.5206 ± 0.0281
Saved final CV model to saved_models/Unigram_CV_svm_pipeline.pkl
Bigram - Fold 1 Accuracy: 0.5000
Bigram - Fold 2 Accuracy: 0.4827
Bigram - Fold 3 Accuracy: 0.5025
Bigram - Fold 4

In [10]:
# Reload all models into a new dictionary
loaded_pipelines = {}
for name in embedding_experiments.keys():
    model_variants = {}
    # Load train/test split model
    path_train = os.path.join(save_dir, f"{name}_svm_pipeline.pkl")
    if os.path.exists(path_train):
        model_variants['train'] = joblib.load(path_train)
        print(f"Loaded trained model: {name}")

    # Load final CV model
    path_cv = os.path.join(save_dir, f"{name}_CV_svm_pipeline.pkl")
    if os.path.exists(path_cv):
        model_variants['cv'] = joblib.load(path_cv)
        print(f"Loaded CV model: {name}")

    # Optional: use loaded_pipelines as default in your prediction functions
    loaded_pipelines[name] = model_variants


Loaded trained model: POS
Loaded CV model: POS
Loaded trained model: Unigram
Loaded CV model: Unigram
Loaded trained model: Bigram
Loaded CV model: Bigram
Loaded trained model: Trigram
Loaded CV model: Trigram


In [39]:
def get_pos_features(sentence):
    return np.random.rand(50)

def get_unigram_features(sentence):
    return np.random.rand(100)

def get_bigram_features(sentence):
    return np.random.rand(50)

def get_trigram_features(sentence):
    return np.random.rand(50)

def extract_features_for_models(sentence):
    features = {}

    features['POS'] = get_pos_features(sentence)
    features['Unigram'] = get_unigram_features(sentence)
    features['Bigram'] = get_bigram_features(sentence)
    features['Trigram'] = get_trigram_features(sentence)

    return features

def predict_sentence(sentence, pipelines):
    feats_dict = extract_features_for_models(sentence)
    print(f"Input sentence: {sentence}\n")

    for model_name, model_variants in pipelines.items():
        X_test_df = pd.DataFrame({'feat': [feats_dict[model_name]]})

        # Predict using train/test model
        if 'train' in model_variants:
            pred = model_variants['train'].predict(X_test_df)[0]
            prob = model_variants['train'].predict_proba(X_test_df)[0][1]
            print(f"{model_name} (train): {'Correct' if pred == 1 else 'Incorrect'} (confidence: {prob:.2f})")

        # Predict using CV model
        if 'cv' in model_variants:
            pred = model_variants['cv'].predict(X_test_df)[0]
            prob = model_variants['cv'].predict_proba(X_test_df)[0][1]
            print(f"{model_name} (CV): {'Correct' if pred == 1 else 'Incorrect'} (confidence: {prob:.2f})")


# Example usage:
sentence = input("Enter a sentence to check: ")
predict_sentence(sentence, loaded_pipelines)


Enter a sentence to check: she are running now
Input sentence: she are running now

POS (train): Correct (confidence: 1.00)
POS (CV): Correct (confidence: 1.00)
Unigram (train): Incorrect (confidence: 0.00)
Unigram (CV): Incorrect (confidence: 0.00)
Bigram (train): Incorrect (confidence: 0.94)
Bigram (CV): Incorrect (confidence: 0.99)
Trigram (train): Incorrect (confidence: 0.99)
Trigram (CV): Incorrect (confidence: 0.62)
